In [4]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import datetime as dt
import humanize

In [5]:
def hummanizing(x):
    if x < 1:
        return humanize.precisedelta(dt.timedelta(seconds=x),minimum_unit="milliseconds").replace('milliseconds','ms')
    elif x < 60:
        return humanize.precisedelta(dt.timedelta(seconds=x),minimum_unit="seconds").replace('seconds','s')
    else:
        return humanize.precisedelta(dt.timedelta(seconds=x),minimum_unit="minutes").replace('minutes','m')


In [153]:
benches = pd.read_json('results.json')
benches['text'] = benches['seconds'].apply(lambda x: hummanizing(x))
benches.loc[benches.seconds == 600, "text"] = "Time Out"
benches = benches.rename(columns={"seconds": "Iteration Duration, seconds", "dataset": "Dataset"})

In [211]:
for name, group in benches.groupby(['operation']):
    fig = px.bar(group, 
        title=name,
        x='Dataset', 
        y='Iteration Duration, seconds',
        color='backend',
        barmode='group',
        text="text",
        log_y=True,
        template ="ggplot2",
    )
    fig.update_layout(
    title_x=0.5,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        x=0.3
    ),
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    bargap=0.30,
    bargroupgap=0.10,
    )
    fig.update_traces(textfont_size=20)
    fig.show()